In [1]:
import tensorflow as tf
from tensorflow                              import keras
from tensorflow.keras                        import layers, Sequential
from tensorflow.keras.preprocessing.text     import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.metrics         import confusion_matrix

import pandas as pd
import numpy  as np

import matplotlib.pyplot as plt

import os
import time

import nltk
from nltk.corpus import stopwords

In [2]:
# https://www.kaggle.com/c/nlp-getting-started: NLP Disaster Tweets dataset
df = pd.read_csv("train.csv")

In [3]:
df.shape

(7613, 5)

In [4]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
print((df.target == 1).sum()) # Disaster
print((df.target == 0).sum()) # No Disaster

3271
4342


In [6]:
# Preprocessing
import re
import string

def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)

# https://stackoverflow.com/questions/34293875/how-to-remove-punctuation-marks-from-a-string-in-python-3-x-using-translate/34294022
def remove_punct(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [7]:
pattern = re.compile(r"https?://(\S+|www)\.\S+")

for t in df.text:
    matches = pattern.findall(t)
    for match in matches:
        print(t)
        print(match)
        print(pattern.sub(r"", t))
    if len(matches) > 0:
        break

@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C
t
@bbcmtd Wholesale Markets ablaze 


In [8]:
df["text"] = df.text.map(remove_URL)
df["text"] = df.text.map(remove_punct)

In [9]:
# remove stopwords
nltk.download('stopwords')


# Stop Words: A stop word is a commonly used word (such as “the”, “a”, “an”, “in”) that a search engine
# has been programmed to ignore, both when indexing entries for searching and when retrieving them 
# as the result of a search query.
stop = set(stopwords.words("english"))

# https://stackoverflow.com/questions/5486337/how-to-remove-stop-words-using-nltk-or-python
def remove_stopwords(text):
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)

[nltk_data] Downloading package stopwords to /Users/Andy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
# stop

In [11]:
df["text"] = df.text.map(remove_stopwords)

In [12]:
df.text

0            deeds reason earthquake may allah forgive us
1                   forest fire near la ronge sask canada
2       residents asked shelter place notified officer...
3       13000 people receive wildfires evacuation orde...
4       got sent photo ruby alaska smoke wildfires pou...
                              ...                        
7608    two giant cranes holding bridge collapse nearb...
7609    ariaahrary thetawniest control wild fires cali...
7610                      m194 0104 utc5km volcano hawaii
7611    police investigating ebike collided car little...
7612    latest homes razed northern california wildfir...
Name: text, Length: 7613, dtype: object

In [13]:
from collections import Counter

# Count unique words
def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count


counter = counter_word(df.text)

In [14]:
len(counter)

17971

In [15]:
#counter

In [16]:
counter.most_common(5)

[('like', 345), ('im', 299), ('amp', 298), ('fire', 250), ('get', 229)]

In [17]:
num_unique_words = len(counter)
print(num_unique_words)

17971


### Creating X, y data

In [18]:
X = df.text.to_numpy()
y = df.target.to_numpy()

### Splitting the data into Training and testing

In [91]:
# TODO: Create a test train set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [92]:
#print(X_train)

In [93]:
# Split dataset into training and validation set
train_size = int(X_train.shape[0] * 0.8)

train_df = X_train[:train_size]
val_df   = X_train[train_size:]

#print(train_df)

In [94]:
# split text and labels
train_sentences = train_df
train_labels    = y_train[:train_size]
val_sentences   = val_df
val_labels      = y_train[train_size:]

In [95]:
#train_sentences.shape, val_sentences.shape

In [96]:
test_sentences = X_test
test_labels    = y_test

In [97]:
test_sentences.shape

(2513,)

In [98]:
# Tokenize
# vectorize a text corpus by turning each text into a sequence of integers
tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(train_sentences) # fit only to training

In [99]:
# each word has unique index
word_index = tokenizer.word_index

In [100]:
#word_index

In [101]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
test_sequences  = tokenizer.texts_to_sequences(test_sentences)
val_sequences   = tokenizer.texts_to_sequences(val_sentences)

In [102]:
#print(train_sentences[10:15])
#print(train_sequences[10:15])

In [103]:
# Pad the sequences to have the same length
# Max number of words in a sequence
max_length = 20

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding="post", truncating="post")
val_padded   = pad_sequences(val_sequences, maxlen=max_length, padding="post", truncating="post")

In [104]:
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding="post", truncating="post")

In [105]:
#train_padded.shape, test_padded.shape, val_padded.shape

In [106]:
#train_padded[10]

In [107]:
#print(train_sentences[10])
#print(train_sequences[10])
#print(train_padded[10])

In [108]:
# Check reversing the indices

# flip (key, value)
reverse_word_index = dict([(idx, word) for (word, idx) in word_index.items()])

In [109]:
#reverse_word_index

In [110]:
def decode(sequence):
    return " ".join([reverse_word_index.get(idx, "?") for idx in sequence])

In [111]:
decoded_text = decode(train_sequences[10])

#print(train_sequences[10])
#print(decoded_text)

## Shared Metrics

In [112]:
loss    = keras.losses.BinaryCrossentropy(from_logits=False)
optim   = keras.optimizers.Adam(lr=0.001)
metrics = ["accuracy"]

## Creating the LSTM

In [148]:
no_of_cells = 128

In [149]:
LSTM_model = Sequential()
LSTM_model.add(layers.Embedding(num_unique_words, 32,input_length=max_length))

# The layer will take as input an integer matrix of size (batch, input_length),
# and the largest integer (i.e. word index) in the input should be no larger than num_words (vocabulary size).
# Now model.output_shape is (None, input_length, 32), where `None` is the batch dimension.

LSTM_model.add(layers.LSTM(no_of_cells, dropout=0.1, return_sequences=True))
LSTM_model.add(layers.LSTM(no_of_cells, dropout=0.1, return_sequences=True))
LSTM_model.add(layers.LSTM(no_of_cells, dropout=0.1, return_sequences=True))
LSTM_model.add(layers.LSTM(no_of_cells, dropout=0.1))
LSTM_model.add(layers.Dense(1, activation="sigmoid"))

LSTM_model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 20, 32)            575072    
_________________________________________________________________
lstm_20 (LSTM)               (None, 20, 128)           82432     
_________________________________________________________________
lstm_21 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
lstm_22 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
lstm_23 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 129       
Total params: 1,052,385
Trainable params: 1,052,385
Non-trainable params: 0
___________________________________________

In [150]:
LSTM_model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [151]:
start_time = time.time()

LSTM_model.fit(train_padded, train_labels, epochs=20, validation_data=(val_padded, val_labels), verbose=1)

running_time = time.time() - start_time

Epoch 1/20
128/128 [==============================] - 7s 57ms/step - loss: 0.5640 - accuracy: 0.7218 - val_loss: 0.4946 - val_accuracy: 0.7765
Epoch 2/20
128/128 [==============================] - 7s 53ms/step - loss: 0.2812 - accuracy: 0.9034 - val_loss: 0.5194 - val_accuracy: 0.7745
Epoch 3/20
128/128 [==============================] - 7s 56ms/step - loss: 0.1567 - accuracy: 0.9551 - val_loss: 0.7084 - val_accuracy: 0.7686
Epoch 4/20
128/128 [==============================] - 7s 57ms/step - loss: 0.0992 - accuracy: 0.9701 - val_loss: 0.7695 - val_accuracy: 0.7500
Epoch 5/20
128/128 [==============================] - 7s 57ms/step - loss: 0.0735 - accuracy: 0.9775 - val_loss: 0.8541 - val_accuracy: 0.7676
Epoch 6/20
128/128 [==============================] - 7s 57ms/step - loss: 0.0652 - accuracy: 0.9794 - val_loss: 0.9179 - val_accuracy: 0.7618
Epoch 7/20
128/128 [==============================] - 7s 58ms/step - loss: 0.0581 - accuracy: 0.9794 - val_loss: 0.9594 - val_accuracy: 0.7667

In [152]:
print("The training took: {:.2f} seconds.".format(running_time))

The training took: 151.70 seconds.


# Testing LSTM

In [153]:
# Predictting with the training data
LSTM_predictions = LSTM_model.predict(test_padded)
LSTM_predictions = [1 if p > 0.5 else 0 for p in LSTM_predictions]

In [154]:
#print(test_sentences[10:20])

#print(test_labels[10:20])
#print(LSTM_predictions[10:20])

In [155]:
confusion_matrix(LSTM_predictions, test_labels)

array([[1159,  310],
       [ 287,  757]])

## Creating the GRU

In [156]:
GRU_model = Sequential()
GRU_model.add(layers.Embedding(num_unique_words, 32, input_length=max_length))

GRU_model.add(layers.GRU(no_of_cells, dropout=0.1, return_sequences=True))
GRU_model.add(layers.GRU(no_of_cells, dropout=0.1, return_sequences=True))
GRU_model.add(layers.GRU(no_of_cells, dropout=0.1, return_sequences=True))
GRU_model.add(layers.GRU(no_of_cells, dropout=0.1))
GRU_model.add(layers.Dense(1, activation="sigmoid"))

GRU_model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 20, 32)            575072    
_________________________________________________________________
gru_20 (GRU)                 (None, 20, 128)           62208     
_________________________________________________________________
gru_21 (GRU)                 (None, 20, 128)           99072     
_________________________________________________________________
gru_22 (GRU)                 (None, 20, 128)           99072     
_________________________________________________________________
gru_23 (GRU)                 (None, 128)               99072     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 129       
Total params: 934,625
Trainable params: 934,625
Non-trainable params: 0
_______________________________________________

In [157]:
GRU_model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [158]:
start_time = time.time()
GRU_model.fit(train_padded, train_labels, epochs=20, validation_data=(val_padded, val_labels), verbose=1)
running_time = time.time() - start_time

Epoch 1/20
128/128 [==============================] - 8s 60ms/step - loss: 0.6864 - accuracy: 0.5664 - val_loss: 0.6876 - val_accuracy: 0.5549
Epoch 2/20
128/128 [==============================] - 7s 52ms/step - loss: 0.5584 - accuracy: 0.7248 - val_loss: 0.4997 - val_accuracy: 0.7765
Epoch 3/20
128/128 [==============================] - 7s 54ms/step - loss: 0.2592 - accuracy: 0.9108 - val_loss: 0.6002 - val_accuracy: 0.7833
Epoch 4/20
128/128 [==============================] - 7s 52ms/step - loss: 0.1494 - accuracy: 0.9542 - val_loss: 0.6920 - val_accuracy: 0.7775
Epoch 5/20
128/128 [==============================] - 7s 51ms/step - loss: 0.0977 - accuracy: 0.9728 - val_loss: 0.7289 - val_accuracy: 0.7755
Epoch 6/20
128/128 [==============================] - 6s 49ms/step - loss: 0.0741 - accuracy: 0.9782 - val_loss: 0.6006 - val_accuracy: 0.7706
Epoch 7/20
128/128 [==============================] - 7s 51ms/step - loss: 0.0504 - accuracy: 0.9824 - val_loss: 0.8907 - val_accuracy: 0.7657

In [159]:
print("The training took: {:.2f} seconds.".format(running_time))

The training took: 137.27 seconds.


In [160]:
GRU_predictions = GRU_model.predict(test_padded)
GRU_predictions = [1 if p > 0.5 else 0 for p in GRU_predictions]

In [161]:
#print(train_sentences[10:20])

#print(test_labels[10:20])
#print(GRU_predictions[10:20])

In [162]:
confusion_matrix(GRU_predictions, test_labels)

array([[1224,  359],
       [ 222,  708]])